### Download data

In [10]:
import cdsapi


c = cdsapi.Client()

c.retrieve(
    'reanalysis-era5-single-levels-monthly-means',  # dataset for monthly averages
    {
        'product_type': 'monthly_averaged_reanalysis',
        'variable': [
            '2m_temperature',            # Average temperature at 2 meters
            'total_precipitation',       # Total precipitation
            'volumetric_soil_water_layer_1',  # Soil moisture
            '10m_u_component_of_wind',   # Wind u-component at 10m
            '10m_v_component_of_wind',   # Wind v-component at 10m
            'surface_solar_radiation_downwards'  # Solar radiation
        ],
        'year': [str(year) for year in range(2009, 2024)],  # Last 15 years
        'month': [
            '01', '02', '03', '04', '05', '06', 
            '07', '08', '09', '10', '11', '12'
        ],                              # Months for the entire year
        'time': '00:00',
        'area': [
            38, 68, 6, 97,              # North, West, South, East for India
        ],
        'format': 'netcdf'              # Output format (netCDF is common for climate data)
    },
    'era5_monthly_data_09_24.nc')             # Save as file

2024-11-05 23:36:01,772 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-11-05 23:36:01,773 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative page](https://confluence.ecmwf.int/x/uINmFw) for guidance.
2024-11-05 23:36:01,774 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2024-11-05 23:36:01,775 INFO [2024-09-16T00:00:00] Remember that you need to have an ECMWF account to use the new CDS. **Your old CDS credentials will not work in new CDS!**
2024-11-05 23:36:01,777 WARNING [2024-06-16T

2f34c2fddcdca9791f25bab9864902da.nc:   0%|          | 0.00/28.2M [00:00<?, ?B/s]

'era5_monthly_data_09_24.nc'

#### Save to database

In [1]:
import xarray as xr
import ibis
import pandas as pd
import lonboard

ibis.options.interactive = True

In [2]:
# Open the downloaded hourly data file
ds = xr.open_dataset('era5_monthly_data_09_24.nc')

In [3]:
ds

<xarray.Dataset> Size: 65MB
Dimensions:    (date: 180, latitude: 129, longitude: 117)
Coordinates:
    number     int64 8B ...
  * date       (date) int64 1kB 20090101 20090201 20090301 ... 20231101 20231201
  * latitude   (latitude) float64 1kB 38.0 37.75 37.5 37.25 ... 6.5 6.25 6.0
  * longitude  (longitude) float64 936B 68.0 68.25 68.5 ... 96.5 96.75 97.0
    expver     (date) <U4 3kB ...
Data variables:
    t2m        (date, latitude, longitude) float32 11MB ...
    tp         (date, latitude, longitude) float32 11MB ...
    swvl1      (date, latitude, longitude) float32 11MB ...
    u10        (date, latitude, longitude) float32 11MB ...
    v10        (date, latitude, longitude) float32 11MB ...
    ssrd       (date, latitude, longitude) float32 11MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-05T22:37 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
climate_df = ds.to_dataframe().reset_index()
climate_df['date'] = pd.to_datetime(climate_df['date'].astype(str), format='%Y%m%d')

In [6]:
import duckdb

# Connect to DuckDB database
con = duckdb.connect('db/india_model_1.ddb')

# Load the spatial extension to enable spatial functions like ST_Contains
con.execute("INSTALL spatial; LOAD spatial;")

In [8]:
# Create a view for unique ERA5 lat/lon points
con.execute("""
    CREATE VIEW unique_era5_points AS
    SELECT DISTINCT latitude, longitude
    FROM climate_df;
""")

In [9]:
# Create the point-to-region mapping table using the view
con.execute("""
    CREATE TABLE point_to_region_mapping AS
    SELECT p.latitude, p.longitude, t.GID_3, t.NAME_3, t.GID_2, t.NAME_2, t.GID_1, t.NAME_1
    FROM unique_era5_points AS p
    LEFT JOIN ADM_ADM_3 AS t
    ON ST_Contains(t.geometry, ST_Point(p.longitude, p.latitude));
""")

CatalogException: Catalog Error: Table with name ADM_ADM_3 does not exist!
Did you mean "sqlite_temp_master or unique_era5_points"?

LINE 5:     LEFT JOIN ADM_ADM_3 AS t
                      ^

In [44]:
# Create a view that joins climate data with region information
con.execute("""
    CREATE VIEW climate_data_by_region AS
    SELECT c.*,
           m.GID_3, m.NAME_3, m.GID_2, m.NAME_2, m.GID_1, m.NAME_1
    FROM climate_data AS c
    LEFT JOIN point_to_region_mapping AS m
    ON c.latitude = m.latitude AND c.longitude = m.longitude
    WHERE m.GID_3 IS NOT NULL
""")

In [43]:
# Drop the existing view if it exists
con.execute("DROP VIEW IF EXISTS climate_data_by_region")

In [9]:
# Connect to DB
con = ibis.duckdb.connect('db/india_model.ddb', extensions = ['spatial'])

In [23]:
con.list_tables()

['ADM_ADM_0',
 'ADM_ADM_1',
 'ADM_ADM_2',
 'ADM_ADM_3',
 'climate_data',
 'climate_data_by_region',
 'climate_data_with_districts',
 'point_to_region_mapping',
 'unique_era5_points']

In [17]:
import lonboard

In [49]:
t = con.table('climate_data_by_region')
t

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ date                ┃ latitude ┃ longitude ┃ number ┃ expver ┃ t2m        ┃ tp       ┃ swvl1    ┃ u10       ┃ v10       ┃ ssrd         ┃ point                 ┃ GID_3         ┃ NAME_3    ┃ GID_2       ┃ NAME_2       ┃ GID_1    ┃ NAME_1            ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ timestamp(9)        │ float64  │ float64   │ int64  │ string │ float32    │ float32  │ float32  │ float32   │ float32   │ float32      │ geospatial:geometry   │ string        │ string    │ string      │ string       │ string   │ string            │
├─────────────────────┼──────────┼───────────┼────────┼────────┼────────────┼──────────┼──────────┼───────────┼───────────┼──────────────┼───────────────────────┼───────────────┼───────────┼─────────────┼──────────────┼──────────┼───────────────────┤
│ 2012-07-01 00:00:00 │    35.25 │     77.50 │      0 │ 0001   │ 267.140869 │ 0.000246 │ 0.225364 │  0.521592 │ -0.367738 │ 3.056077e+07 │ <POINT (77.5 35.25)>  │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    35.25 │     77.75 │      0 │ 0001   │ 270.275635 │ 0.000242 │ 0.183127 │  0.983506 │  0.149840 │ 2.989414e+07 │ <POINT (77.75 35.25)> │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    35.25 │     78.00 │      0 │ 0001   │ 272.779541 │ 0.000275 │ 0.173774 │  1.265244 │  0.700133 │ 2.951782e+07 │ <POINT (78 35.25)>    │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    35.00 │     77.25 │      0 │ 0001   │ 270.109619 │ 0.000299 │ 0.364722 │ -0.238173 │ -0.765687 │ 2.892134e+07 │ <POINT (77.25 35)>    │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    35.00 │     77.50 │      0 │ 0001   │ 270.629150 │ 0.000292 │ 0.340033 │ -0.162490 │ -0.552308 │ 2.911437e+07 │ <POINT (77.5 35)>     │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    35.00 │     77.75 │      0 │ 0001   │ 271.080322 │ 0.000326 │ 0.311591 │ -0.019423 │ -0.161683 │ 2.922803e+07 │ <POINT (77.75 35)>    │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    35.00 │     78.00 │      0 │ 0001   │ 271.472900 │ 0.000467 │ 0.267981 │  0.510850 │  0.372497 │ 2.925466e+07 │ <POINT (78 35)>       │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    34.75 │     74.25 │      0 │ 0001   │ 287.480713 │ 0.004389 │ 0.352073 │  0.083116 │ -0.075257 │ 2.423194e+07 │ <POINT (74.25 34.75)> │ Z01.14.12.1_1 │ n.a. ( 2) │ Z01.14.12_1 │ Kupwara      │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    34.75 │     74.50 │      0 │ 0001   │ 286.588135 │ 0.002911 │ 0.330985 │  0.041123 │ -0.037171 │ 2.441984e+07 │ <POINT (74.5 34.75)>  │ Z01.14.12.1_1 │ n.a. ( 2) │ Z01.14.12_1 │ Kupwara      │ Z01.14_1 │ Jammu and Kashmir │
│ 2012-07-01 00:00:00 │    34.75 │     76.50 │      0 │ 0001   │ 279.220947 │ 0.001089 │ 0.190299 │  0.123643 │ -0.347230 │ 2.596557e+07 │ <POINT (76.5 34.75)>  │ Z01.14.13.1_1 │ n.a. ( 1) │ Z01.14.13_1 │ Leh (Ladakh) │ Z01.14_1 │ Jammu and Kashmir │
│ …                   │        … │         … │      … │ …      │          … │        … │        … │         … │         … │            … │ …                     │ …             │ …         │ …           │ …            │ …        │ …   

In [50]:
lonboard.viz(t)

/home/denis/.cache/pypoetry/virtualenvs/india-climate-agri-database-dWLTZUkq-py3.12/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

## Link data

In [9]:
import ibis
import lonboard

ibis.options.interactive = True

In [47]:
# Connect to DB
con = ibis.duckdb.connect('db/india_model.ddb', extensions = ['spatial'])

In [29]:
con.list_tables()

['ADM_ADM_0',
 'ADM_ADM_1',
 'ADM_ADM_2',
 'ADM_ADM_3',
 'climate_data',
 'climate_data_by_region',
 'climate_data_with_districts',
 'point_to_region_mapping',
 'unique_era5_points']

In [39]:
con.table('climate_data')

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃ date                ┃ latitude ┃ longitude ┃ number ┃ expver ┃ t2m        ┃ tp       ┃ swvl1    ┃ u10       ┃ v10       ┃ ssrd         ┃ point               ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ timestamp(9)        │ float64  │ float64   │ int64  │ string │ float32    │ float32  │ float32  │ float32   │ float32   │ float32      │ geospatial:geometry │
├─────────────────────┼──────────┼───────────┼────────┼────────┼────────────┼──────────┼──────────┼───────────┼───────────┼──────────────┼─────────────────────┤
│ 2009-01-01 00:00:00 │     38.0 │     68.00 │      0 │ 0001   │ 277.231384 │ 0.001331 │ 0.332615 │ -0.864261 │  0.268668 │ 8.384512e+06 │ <POINT (68 38)>     │
│ 2009-01-01 00:00:00 │     38.0 │     68.25 │      0 │ 0001   │ 276.663025 │ 0.001937 │ 0.325382 │ -0.652835 │ -0.024789 │ 8.363008e+06 │ <POINT (68.25 38)>  │
│ 2009-01-01 00:00:00 │     38.0 │     68.50 │      0 │ 0001   │ 277.217712 │ 0.001918 │ 0.341892 │ -0.188479 │ -0.416879 │ 8.260608e+06 │ <POINT (68.5 38)>   │
│ 2009-01-01 00:00:00 │     38.0 │     68.75 │      0 │ 0001   │ 277.536072 │ 0.001812 │ 0.354267 │  0.037107 │ -0.877816 │ 8.176640e+06 │ <POINT (68.75 38)>  │
│ 2009-01-01 00:00:00 │     38.0 │     69.00 │      0 │ 0001   │ 277.125916 │ 0.001613 │ 0.341449 │ -0.784671 │ -0.818246 │ 8.110080e+06 │ <POINT (69 38)>     │
│ 2009-01-01 00:00:00 │     38.0 │     69.25 │      0 │ 0001   │ 277.079041 │ 0.002290 │ 0.361194 │ -0.111331 │ -0.950570 │ 8.222720e+06 │ <POINT (69.25 38)>  │
│ 2009-01-01 00:00:00 │     38.0 │     69.50 │      0 │ 0001   │ 276.530212 │ 0.002611 │ 0.369693 │ -0.426272 │ -0.682504 │ 8.436736e+06 │ <POINT (69.5 38)>   │
│ 2009-01-01 00:00:00 │     38.0 │     69.75 │      0 │ 0001   │ 275.141541 │ 0.002796 │ 0.371570 │ -0.933108 │ -0.304086 │ 8.851456e+06 │ <POINT (69.75 38)>  │
│ 2009-01-01 00:00:00 │     38.0 │     70.00 │      0 │ 0001   │ 272.657166 │ 0.003097 │ 0.371051 │ -1.045901 │ -0.027719 │ 9.189376e+06 │ <POINT (70 38)>     │
│ 2009-01-01 00:00:00 │     38.0 │     70.25 │      0 │ 0001   │ 269.862244 │ 0.003202 │ 0.347065 │ -1.169925 │  0.094352 │ 1.004851e+07 │ <POINT (70.25 38)>  │
│ …                   │        … │         … │      … │ …      │          … │        … │        … │         … │         … │            … │ …                   │
└─────────────────────┴──────────┴───────────┴────────┴────────┴────────────┴──────────┴──────────┴───────────┴───────────┴──────────────┴─────────────────────┘

In [40]:
con.table('climate_data_by_region')

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┓
┃ date                ┃ latitude ┃ longitude ┃ number ┃ expver ┃ t2m        ┃ tp       ┃ swvl1    ┃ u10       ┃ v10       ┃ ssrd         ┃ point                ┃ GID_3        ┃ NAME_3     ┃ GID_2      ┃ NAME_2        ┃ GID_1    ┃ NAME_1  ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━┩
│ timestamp(9)        │ float64  │ float64   │ int64  │ string │ float32    │ float32  │ float32  │ float32   │ float32   │ float32      │ geospatial:geometry  │ string       │ string     │ string     │ string        │ string   │ string  │
├─────────────────────┼──────────┼───────────┼────────┼────────┼────────────┼──────────┼──────────┼───────────┼───────────┼──────────────┼──────────────────────┼──────────────┼────────────┼────────────┼───────────────┼──────────┼─────────┤
│ 2011-01-01 00:00:00 │     24.5 │     90.50 │      0 │ 0001   │ 289.479614 │ 0.000292 │ 0.307126 │  0.716167 │ -0.597644 │ 1.615258e+07 │ <POINT (90.5 24.5)>  │ NULL         │ NULL       │ NULL       │ NULL          │ NULL     │ NULL    │
│ 2011-01-01 00:00:00 │     24.5 │     90.75 │      0 │ 0001   │ 289.657349 │ 0.000238 │ 0.260602 │  0.550640 │ -0.493640 │ 1.609114e+07 │ <POINT (90.75 24.5)> │ NULL         │ NULL       │ NULL       │ NULL          │ NULL     │ NULL    │
│ 2011-01-01 00:00:00 │     24.5 │     91.25 │      0 │ 0001   │ 289.979614 │ 0.000292 │ 0.222349 │  0.190777 │ -0.283191 │ 1.595392e+07 │ <POINT (91.25 24.5)> │ NULL         │ NULL       │ NULL       │ NULL          │ NULL     │ NULL    │
│ 2011-01-01 00:00:00 │     24.5 │     91.50 │      0 │ 0001   │ 289.655396 │ 0.000322 │ 0.229276 │ -0.033344 │ -0.294909 │ 1.593446e+07 │ <POINT (91.5 24.5)>  │ NULL         │ NULL       │ NULL       │ NULL          │ NULL     │ NULL    │
│ 2011-01-01 00:00:00 │     24.5 │     91.75 │      0 │ 0001   │ 289.778442 │ 0.000324 │ 0.231473 │ -0.174458 │ -0.467761 │ 1.593549e+07 │ <POINT (91.75 24.5)> │ NULL         │ NULL       │ NULL       │ NULL          │ NULL     │ NULL    │
│ 2011-01-01 00:00:00 │     24.5 │     92.25 │      0 │ 0001   │ 289.885864 │ 0.000309 │ 0.264814 │ -0.121723 │ -0.291003 │ 1.589862e+07 │ <POINT (92.25 24.5)> │ IND.4.18.1_1 │ Karimganj  │ IND.4.18_1 │ Karimganj     │ IND.4_1  │ Assam   │
│ 2011-01-01 00:00:00 │     24.5 │     92.50 │      0 │ 0001   │ 289.909302 │ 0.000299 │ 0.297498 │  0.032085 │  0.082532 │ 1.592013e+07 │ <POINT (92.5 24.5)>  │ IND.4.18.1_1 │ Karimganj  │ IND.4.18_1 │ Karimganj     │ IND.4_1  │ Assam   │
│ 2011-01-01 00:00:00 │     24.5 │     92.75 │      0 │ 0001   │ 289.542114 │ 0.000397 │ 0.281812 │  0.028667 │  0.256848 │ 1.590682e+07 │ <POINT (92.75 24.5)> │ IND.4.13.1_1 │ Hailakandi │ IND.4.13_1 │ Hailakandi    │ IND.4_1  │ Assam   │
│ 2011-01-01 00:00:00 │     24.5 │     93.00 │      0 │ 0001   │ 289.067505 │ 0.000397 │ 0.314054 │ -0.196918 │  0.375012 │ 1.591194e+07 │ <POINT (93 24.5)>    │ IND.4.4.1_1  │ Silchar    │ IND.4.4_1  │ Cachar        │ IND.4_1  │ Assam   │
│ 2011-01-01 00:00:00 │     24.5 │     93.25 │      0 │ 0001   │ 287.512817 │ 0.000536 │ 0.323987 │ -0.267231 │  0.216809 │ 1.581568e+07 │ <POINT (93.25 24.5)> │ IND.21.3.4_1 │ Thanlon    │ IND.21.3_1 │ Churachandpur │ IND.21_1 │ Manipur │
│ …                   │        … │         … │      … │ …      │          … │        … │        … │         … │         … │            … │ …                    │ …            │ …          │ …          │ …             │ …        │ …       │
└─────────────────────┴──────────┴───────────┴────────┴────────┴────────────┴──────────┴──────────┴───────────┴───────────┴──────────────┴──────────────────────

In [21]:
con.drop_table('wind_v_component_10m')

In [22]:
climate_tables = con.list_tables(database='climate')
climate_tables

['soil_water_layer1',
 'surface_solar_radiation_downwards',
 'temperature_2m',
 'total_precipitation',
 'wind_u_component_10m',
 'wind_v_component_10m']

In [6]:
con.table('total_precipitation')

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━┓
┃ date     ┃ latitude ┃ longitude ┃ number ┃ expver ┃ tp       ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━┩
│ int64    │ float64  │ float64   │ int64  │ string │ float32  │
├──────────┼──────────┼───────────┼────────┼────────┼──────────┤
│ 20090101 │     38.0 │     68.00 │      0 │ 0001   │ 0.001331 │
│ 20090101 │     38.0 │     68.25 │      0 │ 0001   │ 0.001937 │
│ 20090101 │     38.0 │     68.50 │      0 │ 0001   │ 0.001918 │
│ 20090101 │     38.0 │     68.75 │      0 │ 0001   │ 0.001812 │
│ 20090101 │     38.0 │     69.00 │      0 │ 0001   │ 0.001613 │
│ 20090101 │     38.0 │     69.25 │      0 │ 0001   │ 0.002290 │
│ 20090101 │     38.0 │     69.50 │      0 │ 0001   │ 0.002611 │
│ 20090101 │     38.0 │     69.75 │      0 │ 0001   │ 0.002796 │
│ 20090101 │     38.0 │     70.00 │      0 │ 0001   │ 0.003097 │
│ 20090101 │     38.0 │     70.25 │      0 │ 0001   │ 0.003202 │
│        … │        … │         … │      … │ …      │        … │
└──────────┴──────────┴───────────┴────────┴────────┴──────────┘

In [8]:
t = con.table('temperature_2m', database='climate')

In [10]:
date_column_as_date = t.date.cast("string").substr(0, 4) + '-' + \
                      t.date.cast("string").substr(4, 2) + '-' + \
                      t.date.cast("string").substr(6, 2)

In [57]:
date_column_as_date

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ StringConcat((StringConcat((StringConcat((StringConcat((Substring(Cast(date, string), 0, 4), '-')), Substring(Cast(date, string), 4, 2))), '-')), Substring(Cast(date, string), 6, 2))) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string                                                                                                                                                                                  │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ 2009-01-01                                                                                                                                                                              │
│ …                                                                                                                                                                                       │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [11]:
t = t.mutate(
    point = t.longitude.point(t.latitude),
    date=date_column_as_date
    )

In [13]:
taluks = con.table('ADM_ADM_3')
taluks

┏━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GID_3       ┃ GID_0  ┃ COUNTRY ┃ GID_1   ┃ NAME_1              ┃ NL_NAME_1 ┃ GID_2     ┃ NAME_2                   ┃ NL_NAME_2 ┃ NAME_3       ┃ VARNAME_3 ┃ NL_NAME_3 ┃ TYPE_3 ┃ ENGTYPE_3 ┃ CC_3   ┃ HASC_3 ┃ geometry                                                                         ┃
┡━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ string      │ string │ string  │ string  │ string              │ string    │ string    │ string                   │ string    │ string       │ string    │ string    │ string │ string    │ string │ string │ geospatial:geometry                                                              │
├─────────────┼────────┼─────────┼─────────┼─────────────────────┼───────────┼───────────┼──────────────────────────┼───────────┼──────────────┼───────────┼───────────┼────────┼───────────┼────────┼────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ IND.1.1.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.1_1 │ Nicobar Islands          │ NA        │ n.a. ( 2304) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((93.79 6.852, 93.79 6.852, 93.791 6.852, 93.791 6.851, 93.79...> │
│ IND.1.2.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.2_1 │ North and Middle Andaman │ NA        │ n.a. ( 2178) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((92.844 12.15, 92.845 12.15, 92.845 12.15, 92.845 12.15, 92....> │
│ IND.1.3.1_1 │ IND    │ India   │ IND.1_1 │ Andaman and Nicobar │ NA        │ IND.1.3_1 │ South Andaman            │ NA        │ n.a. ( 2178) │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((92.521 10.897, 92.523 10.895, 92.523 10.895, 92.526 10.891,...> │
│ IND.2.1.1_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Anantapur    │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.845 14.586, 77.839 14.588, 77.836 14.589, 77.83 14.591, ...> │
│ IND.2.1.2_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Dharmavaram  │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.355 14.522, 77.355 14.533, 77.375 14.536, 77.388 14.545,...> │
│ IND.2.1.3_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Gooty        │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.754 14.889, 77.742 14.893, 77.719 14.904, 77.714 14.91, ...> │
│ IND.2.1.4_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Hindupur     │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.904 14.024, 77.899 14.012, 77.899 14.006, 77.897 14.003,...> │
│ IND.2.1.5_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Kadiri       │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOLYGON (((77.86 14.35, 77.86 14.356, 77.857 14.364, 77.857 14.367, 77...> │
│ IND.2.1.6_1 │ IND    │ India   │ IND.2_1 │ Andhra Pradesh      │ NA        │ IND.2.1_1 │ Anantapur                │ NA        │ Kalyandurg   │ NA        │ NA        │ Taluk  │ Taluk     │ NA     │ NA     │ <MULTIPOL

In [14]:
combined = t.join(
    taluks,
    predicates=[taluks.geometry.contains(t.point)],
    how='left'
).select(
    t,  # Select station fields
    taluks['GID_3'],
    taluks['NAME_3'],
    taluks['GID_2'],
    taluks['NAME_2'],
    taluks['GID_1'],
    taluks['NAME_1'],
    taluks['geometry']
)

In [ ]:
t.join(
    taluks,
    predicates=[taluks.geometry.contains(t.point)],
    how='left'
).describe()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
combined.group_by(['date'])

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ date     ┃ latitude ┃ longitude ┃ number ┃ expver ┃ t2m        ┃ point                 ┃ GID_3        ┃ NAME_3    ┃ GID_2      ┃ NAME_2    ┃ GID_1    ┃ NAME_1            ┃ geometry                                                                         ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64    │ float64  │ float64   │ int64  │ string │ float32    │ point:geometry        │ string       │ string    │ string     │ string    │ string   │ string            │ geospatial:geometry                                                              │
├──────────┼──────────┼───────────┼────────┼────────┼────────────┼───────────────────────┼──────────────┼───────────┼────────────┼───────────┼──────────┼───────────────────┼──────────────────────────────────────────────────────────────────────────────────┤
│ 20090101 │    34.00 │     75.25 │      0 │ 0001   │ 266.682556 │ <POINT (75.25 34)>    │ Z01.14.1.1_1 │ n.a. ( 6) │ Z01.14.1_1 │ Anantnag  │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((75.486 33.41, 75.478 33.398, 75.462 33.398, 75.456 33.387, ...> │
│ 20090101 │    33.75 │     75.25 │      0 │ 0001   │ 270.426697 │ <POINT (75.25 33.75)> │ Z01.14.1.1_1 │ n.a. ( 6) │ Z01.14.1_1 │ Anantnag  │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((75.486 33.41, 75.478 33.398, 75.462 33.398, 75.456 33.387, ...> │
│ 20090101 │    34.00 │     74.50 │      0 │ 0001   │ 271.473572 │ <POINT (74.5 34)>     │ Z01.14.2.1_1 │ n.a. ( 8) │ Z01.14.2_1 │ Badgam    │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.561 33.703, 74.552 33.696, 74.544 33.691, 74.535 33.686,...> │
│ 20090101 │    34.00 │     74.75 │      0 │ 0001   │ 273.565369 │ <POINT (74.75 34)>    │ Z01.14.2.1_1 │ n.a. ( 8) │ Z01.14.2_1 │ Badgam    │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.561 33.703, 74.552 33.696, 74.544 33.691, 74.535 33.686,...> │
│ 20090101 │    33.75 │     74.50 │      0 │ 0001   │ 269.899353 │ <POINT (74.5 33.75)>  │ Z01.14.2.1_1 │ n.a. ( 8) │ Z01.14.2_1 │ Badgam    │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.561 33.703, 74.552 33.696, 74.544 33.691, 74.535 33.686,...> │
│ 20090101 │    34.50 │     74.75 │      0 │ 0001   │ 267.184509 │ <POINT (74.75 34.5)>  │ Z01.14.3.1_1 │ n.a. ( 4) │ Z01.14.3_1 │ Bandipore │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.577 34.769, 74.58 34.77, 74.612 34.746, 74.618 34.742, 7...> │
│ 20090101 │    34.50 │     75.00 │      0 │ 0001   │ 261.362244 │ <POINT (75 34.5)>     │ Z01.14.3.1_1 │ n.a. ( 4) │ Z01.14.3_1 │ Bandipore │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.577 34.769, 74.58 34.77, 74.612 34.746, 74.618 34.742, 7...> │
│ 20090101 │    34.50 │     75.25 │      0 │ 0001   │ 258.766541 │ <POINT (75.25 34.5)>  │ Z01.14.3.1_1 │ n.a. ( 4) │ Z01.14.3_1 │ Bandipore │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.577 34.769, 74.58 34.77, 74.612 34.746, 74.618 34.742, 7...> │
│ 20090101 │    34.50 │     74.50 │      0 │ 0001   │ 268.680603 │ <POINT (74.5 34.5)>   │ Z01.14.4.1_1 │ n.a. ( 4) │ Z01.14.4_1 │ Baramulla │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.255 33.997, 74.249 34.014, 74.247 34.016, 74.237 34.026,...> │
│ 20090101 │    34.25 │     74.25 │      0 │ 0001   │ 270.403259 │ <POINT (74.25 34.25)> │ Z01.14.4.1_1 │ n.a. ( 4) │ Z01.14.4_1 │ Baramulla │ Z01.14_1 │ Jammu and Kashmir │ <MULTIPOLYGON (((74.255 33.997, 74.249 34.014, 74.247 34.016, 74.237 34.026,...> │
│        … │        … │         … │      … │ …      │          … │ …                     │ …            │ …         │ …          │ …         │ … 

In [37]:
lonboard.viz(t)

/home/denis/.cache/pypoetry/virtualenvs/india-climate-agri-database-dWLTZUkq-py3.12/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [53]:
for t in climate_tables:
    table = con.table(t, database='climate')
    updated_table = table.mutate(
        point=table.longitude.point(table.latitude)
    )
    # Save changes to a new table with "_updated" suffix
    con.create_table(
        name = t,
        obj = updated_table,
        overwrite=True
    )
    

ParserException: Parser Error: Expected a constant as type modifier

In [51]:
updated_table

┏━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ date     ┃ latitude ┃ longitude ┃ number ┃ expver ┃ v10       ┃ point              ┃
┡━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ int64    │ float64  │ float64   │ int64  │ string │ float32   │ point:geometry     │
├──────────┼──────────┼───────────┼────────┼────────┼───────────┼────────────────────┤
│ 20090101 │     38.0 │     68.00 │      0 │ 0001   │  0.268668 │ <POINT (68 38)>    │
│ 20090101 │     38.0 │     68.25 │      0 │ 0001   │ -0.024789 │ <POINT (68.25 38)> │
│ 20090101 │     38.0 │     68.50 │      0 │ 0001   │ -0.416879 │ <POINT (68.5 38)>  │
│ 20090101 │     38.0 │     68.75 │      0 │ 0001   │ -0.877816 │ <POINT (68.75 38)> │
│ 20090101 │     38.0 │     69.00 │      0 │ 0001   │ -0.818246 │ <POINT (69 38)>    │
│ 20090101 │     38.0 │     69.25 │      0 │ 0001   │ -0.950570 │ <POINT (69.25 38)> │
│ 20090101 │     38.0 │     69.50 │      0 │ 0001   │ -0.682504 │ <POINT (69.5 38)>  │
│ 20090101 │     38.0 │     69.75 │      0 │ 0001   │ -0.304086 │ <POINT (69.75 38)> │
│ 20090101 │     38.0 │     70.00 │      0 │ 0001   │ -0.027719 │ <POINT (70 38)>    │
│ 20090101 │     38.0 │     70.25 │      0 │ 0001   │  0.094352 │ <POINT (70.25 38)> │
│        … │        … │         … │      … │ …      │         … │ …                  │
└──────────┴──────────┴───────────┴────────┴────────┴───────────┴────────────────────┘

#### Explore the data

In [54]:
import ibis
import lonboard

ibis.options.interactive = True

In [55]:
# Connect to DB
con = ibis.duckdb.connect('db/india_model.ddb', extensions = ['spatial'])

In [56]:
con.list_databases()

['climate', 'information_schema', 'main', 'pg_catalog']

In [57]:
con.list_tables(database='climate')

['soil_water_layer1',
 'surface_solar_radiation_downwards',
 'temperature_2m',
 'total_precipitation',
 'wind_u_component_10m',
 'wind_v_component_10m']

In [59]:
t = con.table('temperature_2m', database='climate')

t.mutate(point=ibis.point(t['longitude'], t['latitude']))

AttributeError: module 'ibis' has no attribute 'point'. 

#### ERA5 Monthly Climate Data Download Summary

This notebook downloads monthly-averaged reanalysis climate data for India from the ERA5 dataset using the CDS API. It retrieves the following variables for each month from 2009 to 2023:

- 2m temperature (average air temperature at 2 meters)
- Total precipitation
- Volumetric soil water layer 1 (topsoil moisture)
- 10m u-component of wind (east-west wind)
- 10m v-component of wind (north-south wind)
- Surface solar radiation downwards

The data covers the full geographic extent of India and is saved in NetCDF format for further processing.